In [1]:

import os
os.getcwd() 
os.environ["TOKENIZERS_PARALLELISM"] = "false"
import warnings
from test import *
warnings.filterwarnings("ignore")
from datasets import load_dataset,load_metric
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import torch_optimizer as optim
from transformers.optimization import Adafactor, AdafactorSchedule
import torch.backends.cudnn as cudnn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler, SubsetRandomSampler
from torch.autograd import Variable
import logging
import sys
import transformers
from basic_model import *
import time
import argparse
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
import string
from cycle import *
from train import *

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
model = torch.load('./model/D_A.pt')

In [3]:
#real sentences
x = [[  644,   229,   319,   218,    67, 22732,    93,   272,  4015, 10244,
            15,     7,   193,   549, 29214,     5,     1,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0],
        [ 7672,     3, 20962,    77,    55,  1674,    56,    67,  9794, 10206,
           311,  1403,   219,  7278,     6,   862,     3,   362,  3310,  2278,
           319,     3, 26618,     6,   561,   170,   340, 20647,   193,  8816,
            29,  1386, 15742,  5895,  7990, 13879,   425,   170,  9003,     5,
             1,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0],
        [ 9654,  5282,   256,  7537,    29, 14236,  9532,  1238,    32, 18676,
          8421,  6163,    35,     5,     1,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0]]


In [4]:
x_ = torch.Tensor(x).long().cuda()
y_ =x_[:,:20]

In [5]:
x_.shape

torch.Size([3, 196])

In [6]:
y_.shape

torch.Size([3, 20])

In [7]:
model(x_)

disshape torch.Size([3, 196, 512])
mean tensor([[ 0.1416,  0.0000,  0.1090,  ..., -0.1015,  0.0227,  0.3774],
        [ 0.0852,  0.3197, -0.0265,  ..., -0.0905,  0.1005,  0.0000],
        [ 0.0240,  0.0000, -0.0180,  ..., -0.1067,  0.0137,  0.2924]],
       device='cuda:0', grad_fn=<SqueezeBackward0>)


tensor([[0.6153],
        [0.7095],
        [0.5452]], device='cuda:0', grad_fn=<ReluBackward0>)

In [8]:
model(y_)

disshape torch.Size([3, 20, 512])
mean tensor([[ 0.2800,  0.2522,  0.2520,  ..., -0.0985, -0.1251,  0.0500],
        [ 0.0292, -0.2700, -0.0000,  ..., -0.2254, -0.0247, -0.0149],
        [ 0.0454,  0.2872,  0.0974,  ..., -0.0000, -0.0420,  0.0310]],
       device='cuda:0', grad_fn=<SqueezeBackward0>)


tensor([[0.],
        [0.],
        [0.]], device='cuda:0', grad_fn=<ReluBackward0>)

## We can see that the D judge the sentence by it's length
## small output for the first part, but large for the latter part, so the longer the higher

## so we change the output to [batch,0,512] ie, onlythe first word's sumQ*V and than droppout and then classifier